Install pre-requirements

In [2]:
!pip install tensorflow==2.3.0rc0

     |████████████████████████████████| 320.4MB 53kB/s 
     |████████████████████████████████| 3.0MB 54.8MB/s 
     |████████████████████████████████| 460kB 51.9MB/s 
     |████████████████████████████████| 20.1MB 1.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


Import libraries

In [4]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from functools import partial

import time
import pathlib
import math
import os
import sys
print(tf.__version__)

2.3.0-rc0


Define Global VARIABLES


In [5]:
SEPCTURAL_SAMPLES = 10
FEATURE_DIM = SEPCTURAL_SAMPLES*6*2
CONV_LEN = 3
CONV_LEN_INTE = 3#4
CONV_LEN_LAST = 3#5
CONV_NUM = 64
CONV_MERGE_LEN = 8
CONV_MERGE_LEN2 = 6
CONV_MERGE_LEN3 = 4
CONV_NUM2 = 64
INTER_DIM = 120
OUT_DIM = 6#len(idDict)
WIDE = 20
CONV_DROP_PROB = 0.2

BATCH_SIZE = 64
TOTAL_ITER_NUM = 1000000000

Load the Dataset

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
TRAINING_FILENAMES = tf.io.gfile.glob("./DeepSenseData/train.tfrecord")
VALID_FILENAMES = tf.io.gfile.glob("./DeepSenseData/eval.tfrecord")
def read_tfrecord(example):
    tfrecord_format = (
        {
            'label': tf.io.FixedLenFeature([OUT_DIM], tf.float32),
            'example': tf.io.FixedLenFeature([WIDE*FEATURE_DIM], tf.float32),
        }
    )
    example = tf.io.parse_single_example(example, tfrecord_format)
    features = tf.expand_dims(example['example'], axis=0)
    features = tf.reshape(features, shape=(WIDE, FEATURE_DIM))
    features = tf.expand_dims(features, axis=2)
    print(features.shape)
    input1, input2 = tf.split(features, num_or_size_splits=2, axis=1)
    print(input1.shape)
    label = example['label']
    return (input1, input2) , label

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(
        filenames
    )  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(
        ignore_order
    )  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(
        partial(read_tfrecord), num_parallel_calls=AUTOTUNE
    )
    # returns a dataset of (image, label) pairs if labeled=True or just images if labeled=False
    return dataset

def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(1192)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

train_dataset = get_dataset(TRAINING_FILENAMES)
valid_dataset = get_dataset(VALID_FILENAMES)


(20, 120, 1)
(20, 60, 1)
(20, 120, 1)
(20, 60, 1)


Define a class for every Sensor input

In [7]:
class Sensor:
  def __init__(self, name, shape, filters1, kernel_size1, strides1, filters2, kernel_size2, strides2, filters3, kernel_size3, strides3):
    self.shape = shape #list of integers
    self.name = name #string with input name

    self.filters1=filters1
    self.kernel_size1=kernel_size1
    self.strides1=strides1

    self.filters2=filters2
    self.kernel_size2=kernel_size2
    self.strides2=strides2

    self.filters3=filters3
    self.kernel_size3=kernel_size3
    self.strides3=strides3

Define the Sensor parameters and the complete network

In [8]:
K = [Sensor("I1",(20,60,1),64,(1,16),(1,4),64,(1,7),(1,1),64,(1,3),(1,1)),Sensor("I2",(20,60,1),64,(1,16),(1,4),64,(1,7),(1,1),64,(1,3),(1,1))]
inputs=[]
sensors_outputs=[]
#from tensorflow.compat import v1
avgNum = np.full((1, 120), 20,dtype=np.float32)

# Create the individual CNN branches
for k in K: 
  inputs.append(keras.Input(shape=(k.shape),name=k.name))

  cnn=layers.Conv2D(k.filters1,k.kernel_size1,k.strides1,"valid", name=k.name+"_conv1")(inputs[-1])
  bn=layers.BatchNormalization(name=k.name+"_BN1")(cnn)
  relu=keras.activations.relu(bn)
  dropout=layers.Dropout(CONV_DROP_PROB,name=k.name+"_dropout1")(relu)

  cnn=layers.Conv2D(k.filters2,k.kernel_size2,k.strides2,"valid", name=k.name+"_conv2")(dropout)
  bn=layers.BatchNormalization(name=k.name+"_BN2")(cnn)
  relu=keras.activations.relu(bn)
  dropout=layers.Dropout(CONV_DROP_PROB,name=k.name+"_dropout2")(relu)

  cnn=layers.Conv2D(k.filters3,k.kernel_size3,k.strides3,"valid", name=k.name+"_conv3")(dropout)
  bn=layers.BatchNormalization(name=k.name+"_BN3")(cnn)
  relu=keras.activations.relu(bn)
  output=layers.Reshape((relu.shape[1], relu.shape[2], 1, relu.shape[3]), name=k.name+"_output")(relu)

  sensors_outputs.append(output)
    
# Concatenate the outputs of all branches
merge=layers.Concatenate(axis=3)(sensors_outputs)
merge=layers.Dropout(CONV_DROP_PROB,name="Merge_dropout1")(merge)
merge = layers.Reshape((merge.shape[1], merge.shape[2]*2, merge.shape[4]))(merge)

# Create the Merge CNN
merge=layers.Conv2D(64,(1,16),(1,1),"same", name="Merge_conv1")(merge)
merge=layers.BatchNormalization(name="Merge_BN1")(merge)
merge=keras.activations.relu(merge)
merge=layers.Dropout(CONV_DROP_PROB,name="Merge_dropout2")(merge)

merge=layers.Conv2D(64,(1,12),(1,1),"same", name="Merge_conv2")(merge)
merge=layers.BatchNormalization(name="Merge_BN2")(merge)
merge=keras.activations.relu(merge)
merge=layers.Dropout(CONV_DROP_PROB,name="Merge_dropout3")(merge)

merge=layers.Conv2D(64,(1,8),(1,1),"same", name="Merge_conv3")(merge)
merge=layers.BatchNormalization(name="Merge_BN3")(merge)
merge=keras.activations.relu(merge)
merge_output=layers.Reshape((merge.shape[1], merge.shape[2]*merge.shape[3]), name="Merge_output")(merge)

# Create the RNN network with 2 Stacked GRUs
rnn=layers.StackedRNNCells([layers.GRUCell(INTER_DIM, dropout=0.5),layers.GRUCell(INTER_DIM,dropout=0.5)])
rnn_output=layers.RNN(rnn,time_major = False,name="GRU")(merge_output)

# Create the final Dense Layer
rnn_output = rnn_output/avgNum
outputs=layers.Dense(6,name="Output",activation="softmax")(rnn_output)

# Define the model
model = keras.Model(inputs=inputs, outputs=outputs, name="DeepSense")

#Print summary and plot the model
model.summary()
keras.utils.plot_model(model, "DeepSense.png", show_shapes=True)


# Compile the model
model.compile(
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    #loss=loss,
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4, beta_1=0.5,	beta_2=0.9),
    metrics=["accuracy"],
)

Model: "DeepSense"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
I1 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I2 (InputLayer)                 [(None, 20, 60, 1)]  0                                            
__________________________________________________________________________________________________
I1_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I1[0][0]                         
__________________________________________________________________________________________________
I2_conv1 (Conv2D)               (None, 20, 12, 64)   1088        I2[0][0]                         
__________________________________________________________________________________________

Train the model

In [9]:
# Create a callback function to stop the training after reaching a  specific accuracy
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    print(logs)
    if(logs.get('val_accuracy')>0.94):
      print("\nReached 94% accuracy so cancelling training!")
      self.model.stop_training = True
callbacks = myCallback()

# Train the model
history = model.fit(
    train_dataset,
    epochs=5000,
    validation_data=valid_dataset,
    verbose = 1,
    callbacks=[callbacks]
)

#Save the model
model.save("DeepSense")

Epoch 1/5000
   1861/Unknown - 373s 201ms/step - loss: 1.6527 - accuracy: 0.7145{'loss': 1.652748942375183, 'accuracy': 0.7144860625267029, 'val_loss': 1.5981473922729492, 'val_accuracy': 0.714165985584259}
1861/1861 [==============================] - 380s 204ms/step - loss: 1.6527 - accuracy: 0.7145 - val_loss: 1.5981 - val_accuracy: 0.7142
Epoch 2/5000
1861/1861 [==============================] - 380s 204ms/step - loss: 1.4540 - accuracy: 0.8782 - val_loss: 1.4650 - val_accuracy: 0.7712
Epoch 3/5000
1861/1861 [==============================] - 377s 203ms/step - loss: 1.3305 - accuracy: 0.9240 - val_loss: 1.4273 - val_accuracy: 0.7393
Epoch 4/5000
1861/1861 [==============================] - 374s 201ms/step - loss: 1.2482 - accuracy: 0.9399 - val_loss: 1.3596 - val_accuracy: 0.7653
Epoch 5/5000
1861/1861 [==============================] - 351s 188ms/step - loss: 1.1935 - accuracy: 0.9465 - val_loss: 1.3211 - val_accuracy: 0.7795
Epoch 6/5000
1861/1861 [==============================] 

Epoch 24/5000
1861/1861 [==============================] - 354s 190ms/step - loss: 1.0592 - accuracy: 0.9844 - val_loss: 1.2374 - val_accuracy: 0.8039
Epoch 25/5000
1861/1861 [==============================] - 353s 190ms/step - loss: 1.0583 - accuracy: 0.9854 - val_loss: 1.2628 - val_accuracy: 0.7787
Epoch 26/5000
1861/1861 [==============================] - 360s 193ms/step - loss: 1.0582 - accuracy: 0.9855 - val_loss: 1.2326 - val_accuracy: 0.8080
Epoch 27/5000
1861/1861 [==============================] - 351s 189ms/step - loss: 1.0567 - accuracy: 0.9869 - val_loss: 1.3031 - val_accuracy: 0.7376
Epoch 28/5000
1861/1861 [==============================] - 355s 191ms/step - loss: 1.0577 - accuracy: 0.9860 - val_loss: 1.2413 - val_accuracy: 0.7997
Epoch 29/5000
1861/1861 [==============================] - 358s 192ms/step - loss: 1.0573 - accuracy: 0.9865 - val_loss: 1.2587 - val_accuracy: 0.7829
Epoch 30/5000
1861/1861 [==============================] - 358s 192ms/step - loss: 1.0577 - ac

Epoch 47/5000
1861/1861 [==============================] - 364s 196ms/step - loss: 1.0534 - accuracy: 0.9902 - val_loss: 1.2058 - val_accuracy: 0.8349
Epoch 48/5000
1861/1861 [==============================] - 366s 196ms/step - loss: 1.0596 - accuracy: 0.9840 - val_loss: 1.2258 - val_accuracy: 0.8173
Epoch 49/5000
1861/1861 [==============================] - 365s 196ms/step - loss: 1.0537 - accuracy: 0.9899 - val_loss: 1.1938 - val_accuracy: 0.8491
Epoch 50/5000
1861/1861 [==============================] - 364s 196ms/step - loss: 1.0579 - accuracy: 0.9856 - val_loss: 1.1840 - val_accuracy: 0.8600
Epoch 51/5000
1861/1861 [==============================] - 365s 196ms/step - loss: 1.0528 - accuracy: 0.9906 - val_loss: 1.1899 - val_accuracy: 0.8533
Epoch 52/5000
1861/1861 [==============================] - 364s 196ms/step - loss: 1.0522 - accuracy: 0.9914 - val_loss: 1.1912 - val_accuracy: 0.8508
Epoch 53/5000
1861/1861 [==============================] - 365s 196ms/step - loss: 1.0530 - ac

1861/1861 [==============================] - 383s 206ms/step - loss: 1.0513 - accuracy: 0.9922 - val_loss: 1.1218 - val_accuracy: 0.9212
Epoch 70/5000
1861/1861 [==============================] - 380s 204ms/step - loss: 1.0511 - accuracy: 0.9925 - val_loss: 1.1428 - val_accuracy: 0.9003
Epoch 71/5000
1861/1861 [==============================] - 384s 206ms/step - loss: 1.0510 - accuracy: 0.9926 - val_loss: 1.1458 - val_accuracy: 0.8952
Epoch 72/5000
1861/1861 [==============================] - 374s 201ms/step - loss: 1.0508 - accuracy: 0.9928 - val_loss: 1.1520 - val_accuracy: 0.8910
Epoch 73/5000
1861/1861 [==============================] - 375s 202ms/step - loss: 1.0502 - accuracy: 0.9934 - val_loss: 1.1387 - val_accuracy: 0.9044
Epoch 74/5000
1861/1861 [==============================] - 366s 196ms/step - loss: 1.0507 - accuracy: 0.9929 - val_loss: 1.1296 - val_accuracy: 0.9128
Epoch 75/5000
1861/1861 [==============================] - 363s 195ms/step - loss: 1.0506 - accuracy: 0.9929

Epoch 92/5000
1861/1861 [==============================] - 358s 192ms/step - loss: 1.0494 - accuracy: 0.9942 - val_loss: 1.1654 - val_accuracy: 0.8793
Epoch 93/5000
1861/1861 [==============================] - 355s 191ms/step - loss: 1.0495 - accuracy: 0.9940 - val_loss: 1.1258 - val_accuracy: 0.9170
Epoch 94/5000
1861/1861 [==============================] - 354s 190ms/step - loss: 1.0494 - accuracy: 0.9941 - val_loss: 1.1390 - val_accuracy: 0.9044
Epoch 95/5000
1861/1861 [==============================] - 356s 191ms/step - loss: 1.0496 - accuracy: 0.9940 - val_loss: 1.1424 - val_accuracy: 0.8994
Epoch 96/5000
1861/1861 [==============================] - 355s 191ms/step - loss: 1.0495 - accuracy: 0.9941 - val_loss: 1.1389 - val_accuracy: 0.9028
Epoch 97/5000
1861/1861 [==============================] - 357s 192ms/step - loss: 1.0498 - accuracy: 0.9937 - val_loss: 1.1508 - val_accuracy: 0.8919
Epoch 98/5000
1861/1861 [==============================] - 356s 191ms/step - loss: 1.0498 - ac

1861/1861 [==============================] - 351s 188ms/step - loss: 1.0491 - accuracy: 0.9945 - val_loss: 1.1821 - val_accuracy: 0.8609
Epoch 115/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0488 - accuracy: 0.9948 - val_loss: 1.1410 - val_accuracy: 0.9019
Epoch 116/5000
1861/1861 [==============================] - 351s 189ms/step - loss: 1.0489 - accuracy: 0.9946 - val_loss: 1.1494 - val_accuracy: 0.8944
Epoch 117/5000
1861/1861 [==============================] - 350s 188ms/step - loss: 1.0494 - accuracy: 0.9941 - val_loss: 1.1284 - val_accuracy: 0.9145
Epoch 118/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0488 - accuracy: 0.9948 - val_loss: 1.1288 - val_accuracy: 0.9137
Epoch 119/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0485 - accuracy: 0.9952 - val_loss: 1.1329 - val_accuracy: 0.9095
Epoch 120/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0489 - accuracy: 

1861/1861 [==============================] - 355s 191ms/step - loss: 1.0482 - accuracy: 0.9953 - val_loss: 1.1405 - val_accuracy: 0.9019
Epoch 137/5000
1861/1861 [==============================] - 353s 190ms/step - loss: 1.0482 - accuracy: 0.9953 - val_loss: 1.1499 - val_accuracy: 0.8919
Epoch 138/5000
1861/1861 [==============================] - 353s 190ms/step - loss: 1.0482 - accuracy: 0.9954 - val_loss: 1.1568 - val_accuracy: 0.8868
Epoch 139/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0481 - accuracy: 0.9955 - val_loss: 1.1432 - val_accuracy: 0.9003
Epoch 140/5000
1861/1861 [==============================] - 354s 190ms/step - loss: 1.0479 - accuracy: 0.9957 - val_loss: 1.1323 - val_accuracy: 0.9111
Epoch 141/5000
1861/1861 [==============================] - 352s 189ms/step - loss: 1.0479 - accuracy: 0.9957 - val_loss: 1.1348 - val_accuracy: 0.9070
Epoch 142/5000
1861/1861 [==============================] - 354s 190ms/step - loss: 1.0478 - accuracy: 

1861/1861 [==============================] - 355s 191ms/step - loss: 1.0472 - accuracy: 0.9964 - val_loss: 1.1192 - val_accuracy: 0.9237
Epoch 159/5000
1861/1861 [==============================] - 354s 190ms/step - loss: 1.0477 - accuracy: 0.9958 - val_loss: 1.1248 - val_accuracy: 0.9187
Epoch 160/5000
1861/1861 [==============================] - 355s 191ms/step - loss: 1.0471 - accuracy: 0.9965 - val_loss: 1.1124 - val_accuracy: 0.9313
Epoch 161/5000
1861/1861 [==============================] - 355s 191ms/step - loss: 1.0473 - accuracy: 0.9963 - val_loss: 1.1183 - val_accuracy: 0.9246
Epoch 162/5000
1861/1861 [==============================] - 356s 191ms/step - loss: 1.0471 - accuracy: 0.9965 - val_loss: 1.1045 - val_accuracy: 0.9380
Epoch 163/5000
1861/1861 [==============================] - 357s 192ms/step - loss: 1.0471 - accuracy: 0.9965 - val_loss: 1.1363 - val_accuracy: 0.9070
Epoch 164/5000
1861/1861 [==============================] - 359s 193ms/step - loss: 1.0472 - accuracy: 

Convert a TF Lite version of the saved model

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model("DeepSense")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()


tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)


594128

Test the TFLite Model

In [11]:
#Load a sample from the validation dataset
for features, labels in valid_dataset.take(1):
    #print(features[0].numpy()[0].shape)
    #print(features[1].numpy()[0].shape)
    #print(labels.numpy()[0])
    samples1=tf.expand_dims(features[0].numpy()[0], axis=0)
    samples2=tf.expand_dims(features[1].numpy()[0], axis=0)
    label=labels.numpy()[0]    
    
#Define the TFLite interpreter and the inputs/outputs
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

interpreter.set_tensor(input_details[0]['index'], samples1)
interpreter.set_tensor(input_details[1]['index'], samples2)

#Run the inference
interpreter.invoke()

output_data = interpreter.get_tensor(output_details[0]['index'])
output = np.argmax(output_data, axis=1)

#Compare the outputs
print(output)
print(np.argmax(label, axis=0))

(20, 60, 1)
(20, 60, 1)
[0. 0. 1. 0. 0. 0.]
[2]
2
